In [11]:
from flask import Flask, request, jsonify
from flask import render_template
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os
from tensorflow import keras
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# Create required directories
if not os.path.exists('uploads'):
    os.makedirs('uploads')

if not os.path.exists('templates'):
    os.makedirs('templates')

# Create Flask application
app = Flask(__name__)

# Load the model
model = keras.models.load_model(r'C:\Users\nikhi\Desktop\archive\lung_cancer_model_fold_1.h5')

@app.route('/')
def index():
    return render_template('index.html')  # This will use the existing index.html from templates folder

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file uploaded'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    try:
        img_path = os.path.join('uploads', file.filename)
        file.save(img_path)
        
        img_array = preprocess_image(img_path)
        predictions = model.predict(img_array)
        
        # Get the class with highest probability
        predicted_class = np.argmax(predictions[0])
        
        # Map class index to name
        class_names = ['Adenocarcinoma', 'Large Cell Carcinoma', 'Normal', 'Squamous Cell Carcinoma']
        class_name = class_names[predicted_class]
        
        # Get probabilities for each class
        probabilities = {
            'Adenocarcinoma': float(predictions[0][0]),
            'Large Cell Carcinoma': float(predictions[0][1]),
            'Normal': float(predictions[0][2]),
            'Squamous Cell Carcinoma': float(predictions[0][3])
        }
        
        os.remove(img_path)
        
        return jsonify({
            'class_name': class_name,
            'confidence': float(predictions[0][predicted_class]),
            'probabilities': probabilities
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Display the URL
display(HTML(f'<h3>Please click here to open the application: '
            f'<a href="http://localhost:5000" target="_blank">http://localhost:5000</a></h3>'))

if __name__ == '__main__':
    app.run(debug=True, port=5000, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [31/Oct/2024 19:58:35] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 5s 5s/step


127.0.0.1 - - [31/Oct/2024 19:59:25] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 103ms/step


...

127.0.0.1 - - [31/Oct/2024 20:00:09] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 100ms/step


127.0.0.1 - - [31/Oct/2024 20:00:26] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 88ms/step


127.0.0.1 - - [31/Oct/2024 20:00:35] "POST /predict HTTP/1.1" 200 -
